In [ ]:
!pip install librosa

In [ ]:
import os
import numpy as np
import librosa
import matplotlib.pyplot as plt
from scipy.ndimage import zoom

In [1]:
#Fill the path where the mp3 files are
streamed_path = '/content/drive/MyDrive/musics/mp3 hits/'
non_streamed_path = '/content/drive/MyDrive/musics/mp3 non hits/'

fs = 44100
nfft = 2048
overlap = 0.75
hop_length = int(nfft * (1 - overlap))
n_bins = 84
mag_exp = 3
cqt_threshold = -60
target_shape = (256,1024)

In [ ]:
# CQT function calculation in dB
def calc_cqt(x,fs,hop_length, n_bins, mag_exp):
    C = librosa.cqt(x, sr=fs, hop_length=hop_length, fmin=None, n_bins=n_bins)
    C_mag = librosa.magphase(C)[0]**mag_exp
    CdB = librosa.core.amplitude_to_db(C_mag ,ref=np.max)
    return CdB

In [ ]:
#We create a thresholded cqt. All values below thres values will be equal to -120
def cqt_thresholded(cqt, thres):
  new_cqt = np.copy(cqt)
  new_cqt[new_cqt < thres] = -120
  return new_cqt

In [2]:
def cqt_generation(sr=fs, hop_length=hop_length, n_bins=n_bins, mag_exp=mag_exp, cqt_threshold=cqt_threshold, target_shape=target_shape, streamed_path=streamed_path, non_streamed_path=non_streamed_path,path_to_store_streamed='/content/drive/MyDrive/musics/new_cqt hits/',path_to_store_non_streamed='content/drive/MyDrive/musics/new_cqt non hits/'):

  streamed_file_names = os.listdir(streamed_path)
  non_streamed_file_names = os.listdir(non_streamed_path)

  #There is 150 data in /musics/mp3 hits and /musics/mp3 non hits
  for i in range(len(streamed_file_names)):
    print(i)

    #For both files, we only take the first 40 seconds
    signal_streamed, _ = librosa.load(streamed_path + streamed_file_names[i], sr=fs, mono=True) #We load the i_th element of mp3 hits and convert it to a mono signal
    signal_non_streamed, _ = librosa.load(non_streamed_path + non_streamed_file_names[i], sr=fs, mono=True) ##We load the i_th element of mp3 non hits and convert it to a mono signal

    #We remove the first ten seconds
    signal_streamed = signal_streamed[10*fs:]
    signal_non_streamed = signal_non_streamed[10*fs:]

    #cqt
    cqt_streamed = calc_cqt(signal_streamed, fs, hop_length, n_bins, mag_exp)
    cqt_non_streamed = calc_cqt(signal_non_streamed, fs, hop_length, n_bins, mag_exp)

    #We call the cqt threshold function
    cqt_streamed = cqt_thresholded(cqt_streamed, cqt_threshold)
    cqt_non_streamed = cqt_thresholded(cqt_non_streamed, cqt_threshold)

    ##Resize the axis of the spectrograms. axis are very long (len(signal)/hop_length) So we resize them in a better shape for CNN
    cqt_streamed = zoom(cqt_streamed, (target_shape[0] / cqt_streamed.shape[0], target_shape[1] / cqt_streamed.shape[1]))
    cqt_non_streamed = zoom(cqt_non_streamed, (target_shape[0] / cqt_non_streamed.shape[0], target_shape[1] / cqt_non_streamed.shape[1]))

    #For some reasons, the results of the 2 previous lines give us a rotated matrix. So we put it in the right rotation
    cqt_streamed = cqt_streamed.T
    cqt_non_streamed = cqt_non_streamed.T

    cqt_streamed = np.rot90(cqt_streamed)
    cqt_non_streamed = np.rot90(cqt_non_streamed)

    #Save cqt - There will be used in a CNN
    plt.imsave(path_to_store_streamed+streamed_file_names[i][:-4]+'.png', cqt_streamed, cmap='coolwarm')
    plt.imsave(path_to_store_non_streamed+non_streamed_file_names[i][:-4]+'.png', cqt_non_streamed, cmap='coolwarm')




In [ ]:
cqt_generation()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
